In [2]:
import pandas as pd 
import numpy as np
import plotly.express as px
import missingno as msno

In [3]:
dta= pd.read_csv(r"/mnt/78AA22ECAA22A71A/K_REPO/Depi_ML_tasks_-_assignments/Weatheraus_repo/archive/weatherAUS.csv")

In [4]:
dta.dropna(inplace=True)
dta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56420 entries, 6049 to 142302
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           56420 non-null  object 
 1   Location       56420 non-null  object 
 2   MinTemp        56420 non-null  float64
 3   MaxTemp        56420 non-null  float64
 4   Rainfall       56420 non-null  float64
 5   Evaporation    56420 non-null  float64
 6   Sunshine       56420 non-null  float64
 7   WindGustDir    56420 non-null  object 
 8   WindGustSpeed  56420 non-null  float64
 9   WindDir9am     56420 non-null  object 
 10  WindDir3pm     56420 non-null  object 
 11  WindSpeed9am   56420 non-null  float64
 12  WindSpeed3pm   56420 non-null  float64
 13  Humidity9am    56420 non-null  float64
 14  Humidity3pm    56420 non-null  float64
 15  Pressure9am    56420 non-null  float64
 16  Pressure3pm    56420 non-null  float64
 17  Cloud9am       56420 non-null  float64
 18  Cloud3p

In [5]:
numi_features= dta.select_dtypes("number")
cati_features= dta.select_dtypes("object")
cati_features.drop(["RainTomorrow"], axis=1, inplace=True)
cati_features.drop(["Date"], axis=1, inplace=True)

target = ["RainTomorrow"]
features= list(numi_features.columns) + list(cati_features.columns)
features

['MinTemp',
 'MaxTemp',
 'Rainfall',
 'Evaporation',
 'Sunshine',
 'WindGustSpeed',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Cloud9am',
 'Cloud3pm',
 'Temp9am',
 'Temp3pm',
 'Location',
 'WindGustDir',
 'WindDir9am',
 'WindDir3pm',
 'RainToday']

## building the pipeline

In [6]:
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression , SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV ,train_test_split

In [7]:
x_train , x_test , y_train , y_test = train_test_split(dta[features], dta[target] ,
        train_size=.8 ,random_state=99 , stratify=dta["RainTomorrow"] )

In [8]:
# 1'st step to scale and encode 
col_scale_encode= ColumnTransformer([("scale numeric data" , StandardScaler(), list(numi_features.columns) ),
                                ("encode catigorical data", OneHotEncoder(), list(cati_features.columns) ) ],
                                 remainder='passthrough' )

col_encode= ColumnTransformer([("encode catigorical data", OneHotEncoder(), list(cati_features.columns) ) ],
                                 remainder='passthrough' )

col_scale= ColumnTransformer([("scale numeric data" , StandardScaler(), list(numi_features.columns) )] )

In [9]:
# 2'nd step to Pipe the transformers , SGD here is just a place holder
pipe = Pipeline( [ ("columns transformation", col_encode),
                    ("model of classification", SGDClassifier()) ] )

In [10]:
# 3'rd step to define the search grid as a list od dicts 
param_grid = [ {"model of classification" : [SGDClassifier()] ,
                "columns transformation" : [col_scale_encode] ,
                   "model of classification__loss": ['log_loss','perceptron'] , 
                    "model of classification__penalty":['l1','l2']},

                {"model of classification": [LogisticRegression()],
                    "columns transformation" : [col_scale_encode] ,
                    "model of classification__penalty":['l1','l2']},

                {"model of classification" : [DecisionTreeClassifier()] ,
                    "columns transformation" : [col_encode] ,
                    "model of classification__criterion": ['log_loss','entropy','gini'],
                        "model of classification__max_depth": [10,20,40,80]  },

                {"model of classification" : [KNeighborsClassifier()],
                  "columns transformation" : [col_scale] ,
                    "model of classification__n_neighbors": [5,8,10,15,20,25,30]}
]

In [11]:
# finally initiat the gread_search as the hall model estimator
grid_search = GridSearchCV(pipe , param_grid , cv=5 , n_jobs=-1)

In [12]:
grid_search.fit(x_train , y_train)

/home/killd00zer/miniconda3/envs/Yd312/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/killd00zer/miniconda3/envs/Yd312/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/killd00zer/miniconda3/envs/Yd312/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/killd00zer/miniconda3/envs/Yd312/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columns transformation',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('encode '
                                                                         'catigorical '
                                                                         'data',
                                                                         OneHotEncoder(),
                                                                         ['Location',
                                                                          'WindGustDir',
                                                                          'WindDir9am',
                                                                          'WindDir3pm',
                                                                          'RainToday'])])),
                                       ('model of classification',
                                        SGDClassifier())]),
             n_jobs=-1,
             param_grid=[{'columns transformation': [ColumnTransformer(r...
                         {'columns transformation': [ColumnTransformer(transformers=[('scale '
                                                                                      'numeric '
                                                                                      'data',
                                                                                      StandardScaler(),
                                                                                      ['MinTemp',
                                                                                       'MaxTemp',
                                                                                       'Rainfall',
                                                                                       'Evaporation',
                                                                                       'Sunshine',
                                                                                       'WindGustSpeed',
                                                                                       'WindSpeed9am',
                                                                                       'WindSpeed3pm',
                                                                                       'Humidity9am',
                                                                                       'Humidity3pm',
                                                                                       'Pressure9am',
                                                                                       'Pressure3pm',
                                                                                       'Cloud9am',
                                                                                       'Cloud3pm',
                                                                                       'Temp9am',
                                                                                       'Temp3pm'])])],
                          'model of classification': [KNeighborsClassifier()],
                          'model of classification__n_neighbors': [5, 8, 10, 15,
                                                                   20, 25,
                                                                   30]}])

In [13]:
grid_search.best_params_

{'columns transformation': ColumnTransformer(remainder='passthrough',
                   transformers=[('scale numeric data', StandardScaler(),
                                  ['MinTemp', 'MaxTemp', 'Rainfall',
                                   'Evaporation', 'Sunshine', 'WindGustSpeed',
                                   'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
                                   'Humidity3pm', 'Pressure9am', 'Pressure3pm',
                                   'Cloud9am', 'Cloud3pm', 'Temp9am',
                                   'Temp3pm']),
                                 ('encode catigorical data', OneHotEncoder(),
                                  ['Location', 'WindGustDir', 'WindDir9am',
                                   'WindDir3pm', 'RainToday'])]),
 'model of classification': LogisticRegression(),
 'model of classification__penalty': 'l2'}

In [14]:
grid_search.score(x_test,y_test)

0.8563452676355903

In [34]:
results= pd.DataFrame(grid_search.cv_results_).sort_values("mean_test_score").sort_values("mean_test_score")
cols= results.columns

In [35]:
for i in ['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_columns transformation',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score',
       'std_test_score','params'],:
    results.drop(i,axis=1,inplace=True)

In [36]:
results.sort_values(by="rank_test_score", inplace=True )
results

,param_model of classification,param_model of classification__loss,param_model of classification__penalty,param_model of classification__criterion,param_model of classification__max_depth,param_model of classification__n_neighbors,mean_test_score,rank_test_score
5,LogisticRegression(),NaN,l2,NaN,NaN,NaN,0.856744,1
0,SGDClassifier(),log_loss,l1,NaN,NaN,NaN,0.854595,2
1,SGDClassifier(),log_loss,l2,NaN,NaN,NaN,0.852778,3
24,KNeighborsClassifier(),NaN,NaN,NaN,NaN,30.0,0.850873,4
23,KNeighborsClassifier(),NaN,NaN,NaN,NaN,25.0,0.850563,5
21,KNeighborsClassifier(),NaN,NaN,NaN,NaN,15.0,0.850319,6
22,KNeighborsClassifier(),NaN,NaN,NaN,NaN,20.0,0.850009,7
20,KNeighborsClassifier(),NaN,NaN,NaN,NaN,10.0,0.848148,8
19,KNeighborsClassifier(),NaN,NaN,NaN,NaN,8.0,0.846929,9
18,KNeighborsClassifier(),NaN,NaN,NaN,NaN,5.0,0.842432,10
